# Sentiment Analyisis: Ajuste de Hiperparámetros 2

Veremos cómo usar LogisticRegressionCV para hacer ajuste de parámetros.

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
#Lectura de datos de entrenamiento

In [2]:
from util import load_datasets
train, dev, test = load_datasets()
X_train, y_train = train
X_dev, y_dev = dev
X_test, y_test = test

In [ ]:
#Cantidad de datos de entrenamiento

In [3]:
len(train[0])+len(dev[0])+len(test[0])

28070

In [4]:
#Entrenamiento del clasificador

In [5]:
%%time

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from util import print_eval

pipeline = Pipeline([
    ('vect', TfidfVectorizer(
        binary=True,
        min_df=4,
        max_df=0.99,
        ngram_range=(1, 2),
        lowercase = True,      
    )),
    ('clf', LinearSVC(loss='squared_hinge',random_state=0,max_iter=10000,fit_intercept=False,)),    
])

scores = cross_val_score(pipeline,X_train,y_train,cv=25,scoring='roc_auc')
print(scores.mean())
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

0.970977568850929


NameError: name 'print_eval' is not defined

In [11]:
# simples casos de prueba 

In [12]:
pipeline.predict(["good film"])

array([1])

In [13]:
pipeline.predict(["not good film"])

array([0])

In [14]:
pipeline.predict(["bad film"])

array([0])

In [15]:
pipeline.predict(["not bad film"])

array([0])

In [16]:
# Evaluo datos de test

In [19]:
print_eval(pipeline, X_test, y_test)

accuracy	0.91

             precision    recall  f1-score   support

        neg       0.91      0.91      0.91      1352
        pos       0.91      0.92      0.92      1455

avg / total       0.91      0.91      0.91      2807

[[1227  125]
 [ 117 1338]]


In [20]:
# lectura de reviews no etiquetadas

In [21]:
import os
data_dir = "review_polarity_competition/test_reviews_sentoken/"
filenames = []
eval_data = []
dirname = os.path.join(data_dir)
for fname in os.listdir(dirname):
    with open(os.path.join(dirname, fname), 'r') as f:
        content = f.read()
        eval_data.append(content)
        filenames.append(fname)

In [22]:
eval_data[1]

'I gave this to my 4 year and 2 year old grandchildren . They absolutely loved it . We watched it everyday for about 5 days . They got a real kick out of their father joining in and knowing the words and the story . It is definitely a family movie .'

In [23]:
# Clasificacione de reviews no etiquetadas

In [24]:
predicted = pipeline.predict(eval_data)
predicted

array([0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,

In [25]:
# Generacion de matriz con documentos y prediccionces

In [26]:
import numpy as np
matrix = np.array([filenames,predicted])
matrix = matrix.transpose()
# matrix

In [27]:
# Guardo matriz en archivo csv para subir a kaggle

In [28]:
import pandas as pd 
df = pd.DataFrame(matrix)
df.columns = ['Id','Category']
df.to_csv("30_.csv",index=False,)